In [1]:
import pathlib
from decouple import config
from replicate.client import Client

In [2]:
NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent
DATA_DIR = REPO_DIR / "data"
GENERATED_DIR = DATA_DIR / "generated"
GENERATED_DIR.mkdir(exist_ok=True, parents=True)

REPLICATE_API_TOKEN = config("REPLICATE_API_TOKEN")
REPLICATE_MODEL = config("REPLICATE_MODEL", default="flylikeapenguin/imagenation-joe-v1")
REPLICATE_MODEL_VERSION = config("REPLICATE_MODEL_VERSION", default="13ef474b3581629940a325ed0c3b3f3408cfccad7da103e494ae81f5bdfb394a")

In [3]:
replicate_client = Client(api_token=REPLICATE_API_TOKEN)

In [4]:
model = f"{REPLICATE_MODEL}:{REPLICATE_MODEL_VERSION}"
prompt = "a photo of TOK adult man dressed up for a library photo shoot"
num_outputs = 2
output_format = "jpg"

input_args = {
    "prompt": prompt,
    "num_outputs": num_outputs,
    "output_format": output_format,
}

In [5]:
rep_model = replicate_client.models.get(REPLICATE_MODEL)
rep_version = rep_model.versions.get(REPLICATE_MODEL_VERSION)

pred = replicate_client.predictions.create(
    version=rep_version,
    input=input_args
)

In [9]:
pred.id

'c8pqqhr6k1rmc0cm5ejb63gjp8'

In [11]:
pred.status

'starting'

In [15]:
# upstash -> qstash

In [13]:
pred_id = "c8pqqhr6k1rmc0cm5ejb63gjp8"
pred_lookup = replicate_client.predictions.get(pred_id)

In [14]:
pred_lookup.status

'succeeded'

In [15]:
pred_urls = pred_lookup.output
pred_urls

['https://replicate.delivery/xezq/LPyQpoWul0qHC12GpCTqcblJtipCVt5qPIfjaph7rdqtSrAKA/out-0.jpg',
 'https://replicate.delivery/xezq/G7RJO96ak06LF1RGuU1QcJA23yY8gfH41s8KLSwTfLqblWBUA/out-1.jpg']

In [16]:
import httpx
import random

session_id = random.randint(1_000, 40_000)
with httpx.Client() as client:
    for i, url in enumerate(pred_urls):
        fname = f"{i}-{session_id}.jpg"
        outpath = GENERATED_DIR / fname
        res = client.get(url)
        res.raise_for_status()
        with open(outpath, 'wb') as f:
            f.write(res.content)